# Inlet Profiles

Chromatographic systems always require some kind of convective flow through the column.

In this lesson, we will:
- Create and connect our first systems of unit operations.
- Define inlet profiles using piecewise cubic polynomials.
- Run CADET and analyze the results.

## Example 1: Flow from `Inlet` to `Outlet`

In a first example, we will look at a simple system with just two unit operations, an [Inlet](https://cadet-process.readthedocs.io/en/latest/reference/generated/CADETProcess.processModel.Inlet.html), and an [Outlet](https://cadet-process.readthedocs.io/en/latest/reference/generated/CADETProcess.processModel.Outlet.html).

```{figure} ./resources/IO.png
:width: 30%
```

We will introduce flow from the `Inlet` to the `Outlet` with a constant flow rate of $Q = 1~mL \cdot s^{-1}$.
In the first section, the concentration is $1.0~mM$, and after $1~min$, it is changed to $0.0~mM$.

```{figure} ./resources/step.png
:width: 30%
```

## 1. Setting up the model

Before we start with specifying the system, we define some local auxiliary variables.
Note that we have to convert all units to SI units.


```{note}
Generally, CADET can be used with any consistent system of units.
However, we strongly recommend converting everything to the SI system.
```

#### Component System

- `ComponentSystem` ensure that all parts of the process have the same number of components.
- Components can be named which automatically adds legends to the plot methods.

For advanced use, see [here](https://cadet-process.readthedocs.io/en/latest/reference/process_model/component_system.html).

In [ ]:
from CADETProcess.processModel import ComponentSystem

component_system = ComponentSystem(1)

## Unit Operations

For an overview of all models in CADET-Process, see [here](https://cadet-process.readthedocs.io/en/latest/reference/process_model/unit_operation_models.html).

Unit operations require:
- the `ComponentSystem`
- as a unique name.

Note that the name string passed in the constructor is later used to reference the unit in the flow sheet for setting `Events` and `OptimizationVariables`.

## Inlet

In CADET, the `Inlet` pseudo unit operation serves as source for the system and is used to create arbitary concentration profiles as boundary conditions (see also [here](https://cadet-process.readthedocs.io/en/latest/reference/generated/CADETProcess.processModel.Inlet.html)).

- Concentration profiles are described using a third degree piecewise polynomial for each component.
- Flow rate can be expressed as a third degree piecewise polynomial.

Here, the flow rate is constant, we can directly set the parameter on the object.

In [ ]:
from CADETProcess.processModel import Inlet

inlet = Inlet(component_system, 'inlet')
inlet.flow_rate = 1e-6

Note that every unit operation model has different model parameters.
To display all parameters, simply print the `parameters` attribute.

In [ ]:
print(inlet.parameters)

## Outlet
The `Outlet` is another pseudo unit operation that serves as sink for the system (see also [here](https://cadet.github.io/master/modelling/unit_operations/outlet)).

In [ ]:
from CADETProcess.processModel import Outlet

outlet = Outlet(component_system, 'outlet')

Note that the `Outlet` unit does not have any model parameters.

In [ ]:
print(outlet.parameters)

## Flow Sheet Connectivity

The `FlowSheet` stores the connectivity between different unit operations.

For more information, see also [here](https://cadet-process.readthedocs.io/en/latest/reference/process_model/flow_sheet.html).

In [ ]:
from CADETProcess.processModel import FlowSheet

flow_sheet = FlowSheet(component_system)

flow_sheet.add_unit(inlet)
flow_sheet.add_unit(outlet)

flow_sheet.add_connection(inlet, outlet)

## Dynamic Events in Process

Dynamic changes of model parameters or flow sheet connections are configure in `Process` class.

For more information, see also [here](https://cadet-process.readthedocs.io/en/latest/user_guide/process_model/process.html).

In [ ]:
from CADETProcess.processModel import Process

process = Process(flow_sheet, 'process')
process.cycle_time = 120

To add an event that changes the value of a parameter, use the `add_event` method which requires the following arguments:
- `name`: Name of the event.
- `parameter_path`: Path of the parameter that is changed in dot notation. E.g. the flow rate of the eluent unit is the parameter `flow_rate` of the `eluent` unit in the `flow_sheet`. Hence, the path is `flow_sheet.eluent.flow_rate`. As previously mentioned, the name of the unit operation is used to reference it, not the variable.
- `state`: Value of the attribute that is changed at Event execution.
- `time`: Time at which the event is executed.

To display all time dependent parameters of an object, use the `section_dependent_parameters` attribute.

In [ ]:
print(inlet.section_dependent_parameters)

Note that also flow sheet connectivity can be added as events. More on that later.

In [ ]:
print(process.flow_sheet.section_dependent_parameters)

In [ ]:
_ = process.add_event('start load', 'flow_sheet.inlet.c', [1], 0)
_ = process.add_event('start wash', 'flow_sheet.inlet.c', [0], 60)

All events can are stored in the events attribute. To visualize the trajectory of the parameter state over the entire cycle, the Process provides a `plot_events()` method.

In [ ]:
_ = process.plot_events()

## 3. Setting up the simulator and running the simulation

To simulate the process, a process simulator needs to be configured.
If no path is specified, CADET-Process will try to autodetect CADET.

In [ ]:
from CADETProcess.simulator import Cadet
process_simulator = Cadet()

If a specific version of CADET is to be used, add the install path to the constructor:

```
process_simulator = Cadet(install_path='/path/to/cadet/executable')
```

To check that everything works correctly, you can call the check_cadet method:

In [ ]:
process_simulator.check_cadet()

Now, run the simulation:

In [ ]:
simulation_results = process_simulator.simulate(process)

## 4. Plotting the results

The simulation_results object contains the solution for the inlet and outlet of every unit operation also provide plot methods.

In [ ]:
_ = simulation_results.solution.inlet.outlet.plot()
_ = simulation_results.solution.outlet.inlet.plot()